In [ ]:
# Import the necessary libraries
from pathlib import Path
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout
from tensorflow.keras import layers, models
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Define a function to create a TensorBoard callback
def create_tensorboard_callback(log_dir, experiment_name):
    log_dir = log_dir + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir,
        histogram_freq=1,
        write_images=True,
        write_graph=True,
        profile_batch=0
    )
    return tensorboard_callback

# Define the directory where the images are located
image_dir = Path("C:/Users/lbvik/Downloads/BodyPartsDataset/")

# Get a list of all jpg image file paths within the directory
filepaths = list(image_dir.glob('**/*.jpg'))

# Extract labels (directory names) corresponding to each image
labels = [p.parent.name for p in filepaths]

# Convert filepaths list to a Pandas Series and name the column 'Filepath'
filepaths = pd.Series(filepaths, name='Filepath').astype(str)

# Convert labels list to a Pandas Series and name the column 'Label'
labels = pd.Series(labels, name='Label')

# Concatenate the filepaths and labels to form a DataFrame
image_df = pd.concat([filepaths, labels], axis=1)

# Split dataframe into train and test (we will get validation in the next chunk)
train_df, test_df = train_test_split(image_df, test_size=0.2, shuffle=True, random_state=42)

# Create a data generator for training data with a specific preprocessing function and set aside 20% for validation
train_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    validation_split=0.2
)

# Create a data generator for test data with a specific preprocessing function
test_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
)

# Set batch size and target size for the images
batch_size_ = 32
target_size_ = (224, 224)

# Initialize and fit the label encoder
label_encoder = LabelEncoder()
label_encoder.fit(train_df['Label'])

# Use the label encoder to transform string labels into numerical values
train_labels_encoded = label_encoder.transform(train_df['Label'])

# One-hot encode the numerical labels for both training and validation data
train_labels_one_hot = to_categorical(train_labels_encoded, num_classes=5)

# Create a generator for training data and apply label encoding
train_data_generator = ImageDataGenerator(rescale=1.0/255.0)
train_df_generator = train_data_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    class_mode='categorical',
    target_size=(224, 224),
    batch_size=batch_size_,
    seed=42,
)

# Use the train generator to read images from the dataframe, preprocess them, and use one-hot encoded labels
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',  # Assuming 'Label' contains one-hot encoded labels
    target_size=target_size_,
    color_mode='rgb',
    class_mode='categorical',  # Set class_mode to 'categorical' for one-hot encoded labels
    batch_size=batch_size_,
    shuffle=True,
    seed=42,
    subset='training'
)

# Use the validation generator to read images from the dataframe, preprocess them, and use one-hot encoded labels
val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',  # Assuming 'Label' contains one-hot encoded labels
    target_size=target_size_,
    color_mode='rgb',
    class_mode='categorical',  # Set class_mode to 'categorical' for one-hot encoded labels
    batch_size=batch_size_,
    shuffle=True,
    seed=42,
    subset='validation'
)

# Use the test generator to read images from the dataframe and preprocess them for testing
test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col=None,  # Set y_col to None since we're not using class labels for testing
    target_size=target_size_,
    color_mode='rgb',
    class_mode=None,  # Set class_mode to None for testing
    batch_size=batch_size_,
    shuffle=False  # Do not shuffle the test data
)

# Create a data augmentation pipeline to increase the diversity of training data
augment = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(224, 224),
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.experimental.preprocessing.RandomFlip("horizontal"),
  layers.experimental.preprocessing.RandomRotation(0.1),
  layers.experimental.preprocessing.RandomZoom(0.1),
  layers.experimental.preprocessing.RandomContrast(0.1),
])

# Load a pre-trained EfficientNetB0 model without the top classification layer
pretrained_model = tf.keras.applications.efficientnet.EfficientNetB0(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='max'
)

# Set the pre-trained model layers as non-trainable
pretrained_model.trainable = False

# Setup a callback to save model's weights during training
checkpoint_path = "bodyparts_classification_model_checkpoint"
checkpoint_callback = ModelCheckpoint(
    checkpoint_path,
    save_weights_only=True,
    monitor="val_accuracy",
    save_best_only=True
)

# Setup a callback to stop training early if validation loss does not improve for 5 consecutive epochs
early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

# Setup a callback to reduce learning rate when validation loss plateaus
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-6
)

# Start building the model by using the input from the pre-trained model
inputs = pretrained_model.input

# Apply data augmentation to the model's inputs
x = augment(inputs)

# Attach a dense layer with 128 nodes to the pre-trained model's output
x = Dense(128, activation='relu')(pretrained_model.output)

# Apply a dropout layer to prevent overfitting
x = Dropout(0.45)(x)

# Attach another dense layer with 256 nodes
x = Dense(256, activation='relu')(x)

# Apply another dropout layer
x = Dropout(0.45)(x)

# Attach the final classification layer with 5 nodes (for 5 categories)
outputs = Dense(5, activation='softmax')(x)

# Combine the defined architecture to build the complete model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model with Adam optimizer, categorical crossentropy as the loss function, and accuracy as the evaluation metric
model.compile(
    optimizer=Adam(0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model using the training data with specified epochs, validation data, and callbacks
history = model.fit(
    train_images,                   # Training data generator
    validation_data=val_images,     # Validation data generator
    epochs=150,
    verbose=1,
    callbacks=[checkpoint_callback, early_stopping, reduce_lr]
)


# Access the training history from the `history` object
training_loss = history.history['loss']
validation_loss = history.history['val_loss']
training_accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']

# Create subplots for loss and accuracy
plt.figure(figsize=(12, 6))

# Plot training and validation loss
plt.subplot(1, 2, 1)
plt.plot(training_loss, label='Training Loss')
plt.plot(validation_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(training_accuracy, label='Training Accuracy')
plt.plot(validation_accuracy, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Display the plots
plt.tight_layout()
plt.show()

In [1]:
# Import the necessary libraries
from pathlib import Path
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout
from tensorflow.keras import layers, models
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
# Import TensorFlow
import tensorflow as tf

# Load an image for prediction from your Google Drive
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
loaded_model = tf.keras.models.load_model("C:/Users/lbvik/Desktop/MiniProject/BodyParts1.h5")

# Fit the LabelEncoder with the class labels
# Replace 'output_classes' with the actual list of class labels used in your model
output_classes = ["eye", "foot", "nose", "ear", "hand"]  # List of class labels
label_encoder = LabelEncoder()
label_encoder.fit(output_classes)

LabelEncoder()

In [2]:
# Replace 'input_image_path' with the path to the image you want to predict on in your Google Drive
input_image_path = "C:/Users/lbvik/Downloads/iey.jpg"
# Load and preprocess the input image
input_img = image.load_img(input_image_path, target_size=(224, 224))
input_img = image.img_to_array(input_img)
input_img = np.expand_dims(input_img, axis=0)
input_img = tf.keras.applications.efficientnet.preprocess_input(input_img)

# Use the trained model to make a prediction
prediction = loaded_model.predict(input_img)

# Map the prediction to the corresponding category label
predicted_label = label_encoder.inverse_transform([np.argmax(prediction)])[0]

print(f"The predicted category for the input image is: {predicted_label}")

1/1 [==============================] - 2s 2s/step
The predicted category for the input image is: eye


In [7]:
#PREDICTION WITH RESIZED FRAME
import cv2
import numpy as np
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

# Load your pre-trained model and label encoder here
model = tf.keras.models.load_model("C:/Users/lbvik/Desktop/MiniProject/BodyParts1.h5")

# Fit the LabelEncoder with the class labels
# Replace 'output_classes' with the actual list of class labels used in your model
output_classes = ["eye", "hand", "nose", "ear", "foot"]  # List of class labels
label_encoder = LabelEncoder()
label_encoder.fit(output_classes)

# Initialize the webcam using OpenCV with your desired resolution
# Replace 'width' and 'height' with the desired resolution
cap = cv2.VideoCapture(1)
width = 1280  # Desired width
height = 720  # Desired height
cap.set(3, width)
cap.set(4, height)

# Create a GUI window
root = tk.Tk()
root.title("Real-time Video Feed")

# Create a label to display real-time video feed without changing popup dimensions
video_label = tk.Label(root)
video_label.pack()

# Create a variable to hold the image reference
video_frame = None

# Function to update the video feed and process frames
def update_video_feed():
    global frame, video_frame
    ret, frame = cap.read()
    if ret:
        # Resize the frame to the input size expected by your model
        frame = cv2.resize(frame, (224, 224))

        # Preprocess the frame
        preprocessed_frame = tf.keras.applications.efficientnet.preprocess_input(frame)

        # Expand dimensions to match the model's input shape
        preprocessed_frame = np.expand_dims(preprocessed_frame, axis=0)

        # Make predictions using your model
        predictions = model.predict(preprocessed_frame)

        # Get the class label with the highest probability
        predicted_class = label_encoder.inverse_transform([np.argmax(predictions)])[0]

        # Overlay the predicted class on the frame with a reduced font size
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.5  # Reduced font size
        font_color = (0, 255, 0)  # Green color
        font_thickness = 1
        cv2.putText(frame, f'Predicted: {predicted_class}', (10, 30), font, font_scale, font_color, font_thickness)

        # Display the frame with the overlay
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = Image.fromarray(frame)
        video_frame = ImageTk.PhotoImage(image=frame)
        video_label.config(image=video_frame)
        video_label.image = video_frame  # Keep a reference to the image

    video_label.after(10, update_video_feed)

update_video_feed()

root.mainloop()

# Release the webcam
cap.release()


1/1 [==============================] - 0s 136ms/step


In [1]:
#Predicting with capturing and storing...
import cv2
import numpy as np
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

# Load your pre-trained model and label encoder here
model = tf.keras.models.load_model("C:/Users/lbvik/Desktop/MiniProject/BodyParts1.h5")

# Fit the LabelEncoder with the class labels
output_classes = ["eye", "hand", "nose", "ear", "foot"]  # List of class labels
label_encoder = LabelEncoder()
label_encoder.fit(output_classes)

# Initialize the webcam using OpenCV with your desired resolution
cap = cv2.VideoCapture(0)
width = 1280  # Desired width
height = 720  # Desired height
cap.set(3, width)
cap.set(4, height)

# Create a GUI window
root = tk.Tk()
root.title("Real-time Video Feed")

# Create a label to display real-time video feed without changing popup dimensions
video_label = tk.Label(root)
video_label.pack()

# Create a variable to hold the image reference
video_frame = None
captured_image = None

# Function to update the video feed and process frames
def update_video_feed():
    global frame, video_frame
    ret, frame = cap.read()
    if ret:
        # Resize the frame to the input size expected by your model
        frame = cv2.resize(frame, (224, 224))

        # Preprocess the frame
        preprocessed_frame = tf.keras.applications.efficientnet.preprocess_input(frame)

        # Expand dimensions to match the model's input shape
        preprocessed_frame = np.expand_dims(preprocessed_frame, axis=0)

        # Make predictions using your model
        predictions = model.predict(preprocessed_frame)

        # Get the class label with the highest probability
        predicted_class = label_encoder.inverse_transform([np.argmax(predictions)])[0]

        # Overlay the predicted class on the frame with a reduced font size
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.5  # Reduced font size
        font_color = (0, 255, 0)  # Green color
        font_thickness = 1
        cv2.putText(frame, f'Predicted: {predicted_class}', (10, 30), font, font_scale, font_color, font_thickness)

        # Display the frame with the overlay
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = Image.fromarray(frame)
        video_frame = ImageTk.PhotoImage(image=frame)
        video_label.config(image=video_frame)
        video_label.image = video_frame  # Keep a reference to the image

    video_label.after(10, update_video_feed)

# Function to capture an image and save it
def capture_image():
    global frame, captured_image
    ret, frame = cap.read()
    if ret:
        # Save the captured image
        cv2.imwrite("captured_image.jpg", frame)
        captured_image = frame
        info_label.config(text="Image captured and saved as 'captured_image.jpg'.")

# Function to predict the captured image
def predict_captured_image():
    global captured_image
    if captured_image is not None:
        try:
            # Resize the captured image to the input size expected by your model
            captured_image = cv2.resize(captured_image, (224, 224))

            # Preprocess the captured image
            preprocessed_image = tf.keras.applications.efficientnet.preprocess_input(captured_image)

            # Expand dimensions to match the model's input shape
            preprocessed_image = np.expand_dims(preprocessed_image, axis=0)

            # Make predictions using your model
            predictions = model.predict(preprocessed_image)

            # Get the class label with the highest probability
            predicted_class = label_encoder.inverse_transform([np.argmax(predictions)])[0]

            # Display the prediction result
            info_label.config(text=f'Predicted: {predicted_class}')
        except Exception as e:
            info_label.config(text="Error predicting the captured image.")
    else:
        info_label.config(text="No captured image to predict.")

# Create a button to capture an image
capture_button = ttk.Button(root, text="Capture Image", command=capture_image)
capture_button.pack()

# Create a button to predict the captured image
predict_button = ttk.Button(root, text="Predict Captured Image", command=predict_captured_image)
predict_button.pack()

# Create a label to display information
info_label = ttk.Label(root, text="")
info_label.pack()

update_video_feed()

root.mainloop()

# Release the webcam
cap.release()


KeyboardInterrupt: 

In [1]:
#Handling with JSON and its Working
import cv2
import numpy as np
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import json

# Load your pre-trained model and label encoder here
model = tf.keras.models.load_model("C:/Users/lbvik/Desktop/MiniProject/BodyParts1.h5")

# Fit the LabelEncoder with the class labels
output_classes = ["eye", "hand", "nose", "ear", "foot"]  # List of class labels
label_encoder = LabelEncoder()
label_encoder.fit(output_classes)

# Load the body part information from the JSON file
with open("C:/Users/lbvik/Desktop/MiniProject/algorithm/bodyy.json", "r") as json_file:
    body_parts_info = json.load(json_file)

# Initialize the webcam using OpenCV with your desired resolution
cap = cv2.VideoCapture(0)
width = 1280  # Desired width
height = 720  # Desired height
cap.set(3, width)
cap.set(4, height)

# Create a GUI window
root = tk.Tk()
root.title("Real-time Video Feed")

# Create a label to display real-time video feed without changing popup dimensions
video_label = tk.Label(root)
video_label.pack()

# Create a label to display information
info_label = ttk.Label(root, text="", font=("Helvetica", 10))  # Reduced font size
info_label.pack()

# Create a variable to hold the image reference
video_frame = None

# Function to display body part information
def display_body_part_info(predicted_class):
    if predicted_class in body_parts_info['human_body']['anatomy_info']:
        info = body_parts_info['human_body']['anatomy_info'][predicted_class]
        function = info.get("function", "Function information not available.")
        info_label.config(text=f'Predicted: {predicted_class}\nFunction: {function}')
    else:
        info_label.config(text=f'Predicted: {predicted_class}\nFunction information not available.')

# Function to update the video feed and process frames
def update_video_feed():
    global frame, video_frame
    ret, frame = cap.read()
    if ret:
        # Resize the frame to the input size expected by your model
        frame = cv2.resize(frame, (224, 224))

        # Preprocess the frame
        preprocessed_frame = tf.keras.applications.efficientnet.preprocess_input(frame)

        # Expand dimensions to match the model's input shape
        preprocessed_frame = np.expand_dims(preprocessed_frame, axis=0)

        # Make predictions using your model
        predictions = model.predict(preprocessed_frame)

        # Get the class label with the highest probability
        predicted_class = label_encoder.inverse_transform([np.argmax(predictions)])[0]

        # Overlay the predicted class on the frame with a reduced font size
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.5  # Reduced font size
        font_color = (0, 255, 0)  # Green color
        font_thickness = 1
        cv2.putText(frame, f'Predicted: {predicted_class}', (10, 30), font, font_scale, font_color, font_thickness)

        # Display the frame with the overlay
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = Image.fromarray(frame)
        video_frame = ImageTk.PhotoImage(image=frame)
        video_label.config(image=video_frame)
        video_label.image = video_frame  # Keep a reference to the image

        # Display body part information if available in the JSON data
        display_body_part_info(predicted_class)

    video_label.after(10, update_video_feed)

update_video_feed()

root.mainloop()

# Release the webcam
cap.release()

1/1 [==============================] - 0s 185ms/step
